In [1]:
from utils import *
import numpy as np
import torch, os
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import GridSearchCV

In [2]:
# def scaling_embeddings(embeddings):
#     for i in range(len(embeddings)):
#         embeddings[i] = StandardScaler().fit_transform(embeddings[i])
#     return embeddings

def reshape_embeddings(embeddings):
    if embeddings.shape[1] == 1:
        embeddings = embeddings.squeeze(1)
    return embeddings

In [3]:
dynahate_labels_train = process_labels(read_labels("dynahate", "train"))
dynahate_labels_dev = process_labels(read_labels("dynahate", "dev"))
dynahate_labels_test = process_labels(read_labels("dynahate", "test"))

latenthatred_labels_train = read_labels("latenthatred", "train")
latenthatred_labels_dev = read_labels("latenthatred", "dev")
latenthatred_labels_test = read_labels("latenthatred", "test")

olid_labels_train = read_labels("olid", "train")
olid_labels_dev = read_labels("olid", "dev")
olid_labels_test = read_labels("olid", "test")

In [4]:
mlp = MLPClassifier()
gridsearch = GridSearchCV(
    mlp,
    param_grid={
        "hidden_layer_sizes": [(128), (128,64)],
        "activation": ["relu"],
        "solver": ["adam"],
        "learning_rate_init": [0.001, 0.0001],
        "learning_rate": ["adaptive"],
        "early_stopping": [True],
        "max_iter": [10000],
    },
    verbose=4,
    n_jobs=os.cpu_count()//3,
    cv=5,
)

In [5]:
dynahate_labels_train_dev = np.concatenate((dynahate_labels_train, dynahate_labels_dev))
latenthatred_labels_train_dev = np.concatenate((latenthatred_labels_train, latenthatred_labels_dev))
olid_labels_train_dev = np.concatenate((olid_labels_train, olid_labels_dev))

In [6]:
bert_dynahate_train_embeddings = reshape_embeddings(bert_dynahate_train_embeddings)
bert_dynahate_dev_embeddings = reshape_embeddings(bert_dynahate_dev_embeddings)
bert_dynahate_test_embeddings = reshape_embeddings(bert_dynahate_test_embeddings)

bert_latenthatred_train_embeddings = reshape_embeddings(bert_latenthatred_train_embeddings)
bert_latenthatred_dev_embeddings = reshape_embeddings(bert_latenthatred_dev_embeddings)
bert_latenthatred_test_embeddings = reshape_embeddings(bert_latenthatred_test_embeddings)

bert_olid_train_embeddings = reshape_embeddings(bert_olid_train_embeddings)
bert_olid_dev_embeddings = reshape_embeddings(bert_olid_dev_embeddings)
bert_olid_test_embeddings = reshape_embeddings(bert_olid_test_embeddings)

### BERT

#### DynaHate

In [7]:
bert_dynahate_train_dev = np.concatenate((bert_dynahate_train_embeddings, bert_dynahate_dev_embeddings))

grid_results = gridsearch.fit(bert_dynahate_train_dev, dynahate_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_dynahate_train_dev)
test_preds = mlp.predict(bert_dynahate_test_embeddings)

computeAllScores(train_dev_preds, test_preds, dynahate_labels_train_dev, dynahate_labels_test, "Results/bert_dynahate")
save_model(mlp, "Saves/bert_dynahate")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6787759291270528
Accuracy Test:  0.6611650485436893
Weighted F1 Train Dev:  0.6787726354930493
Weighted F1 Test:  0.661198277429779
Macro F1 Train Dev:  0.6769347877192148
Macro F1 Test:  0.6577424732764539
Micro F1 Train Dev:  0.6787759291270528
Micro F1 Test:  0.6611650485436893
Weighted Recall Train Dev:  0.6787759291270528
Weighted Recall Test:  0.6611650485436893
Macro Recall Train Dev:  0.6769311718489633
Macro Recall Test:  0.6577740447430872
Micro Recall Train Dev:  0.6787759291270528
Micro Recall Test:  0.6611650485436893
Confusion Matrix Train Dev: 
[[11168  5949]
 [ 5944 13963]]
Confusion Matrix Test: 
[[1156  696]
 [ 700 1568]]


#### LatentHatred

In [10]:
bert_latenthatred_train_dev = np.concatenate((bert_latenthatred_train_embeddings, bert_latenthatred_dev_embeddings))

grid_results = gridsearch.fit(bert_latenthatred_train_dev, latenthatred_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_latenthatred_train_dev)
test_preds = mlp.predict(bert_latenthatred_test_embeddings)

computeAllScores(train_dev_preds, test_preds, latenthatred_labels_train_dev, latenthatred_labels_test, "Results/bert_latenthatred")
save_model(mlp, "Saves/bert_latenthatred")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': 128, 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.6792054624456859
Accuracy Test:  0.6804469273743017
Weighted F1 Train Dev:  0.6544937224909636
Weighted F1 Test:  0.6557211814076632
Macro F1 Train Dev:  0.4385309599791611
Macro F1 Test:  0.442001430604519
Micro F1 Train Dev:  0.6792054624456859
Micro F1 Test:  0.6804469273743017
Weighted Recall Train Dev:  0.6792054624456859
Weighted Recall Test:  0.6804469273743017
Macro Recall Train Dev:  0.4440187856811955
Macro Recall Test:  0.44417387676416514
Micro Recall Train Dev:  0.6792054624456859
Micro Recall Test:  0.6804469273743017
Confusion Matrix Train Dev: 
[[8345 1596    4]
 [2749 2590    8]
 [ 366  445    7]]
Confusion Matrix Test: 
[[2826  520    0]
 [ 924  823    6]
 [ 111  155    5]]


#### OLID

In [9]:
bert_olid_train_dev = np.concatenate((bert_olid_train_embeddings, bert_olid_dev_embeddings))

grid_results = gridsearch.fit(bert_olid_train_dev, olid_labels_train_dev)

best_params = grid_results.best_params_
mlp = grid_results.best_estimator_

print("Best params: ", best_params)

train_dev_preds = mlp.predict(bert_olid_train_dev)
test_preds = mlp.predict(bert_olid_test_embeddings)

computeAllScores(train_dev_preds, test_preds, olid_labels_train_dev, olid_labels_test, "Results/bert_olid")

save_model(mlp, "Saves/bert_olid")

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best params:  {'activation': 'relu', 'early_stopping': True, 'hidden_layer_sizes': (128, 64), 'learning_rate': 'adaptive', 'learning_rate_init': 0.001, 'max_iter': 10000, 'solver': 'adam'}
Accuracy Train Dev:  0.7559667673716012
Accuracy Test:  0.791860465116279
Weighted F1 Train Dev:  0.7394663819048093
Weighted F1 Test:  0.7613552716242301
Macro F1 Train Dev:  0.6927135347621439
Macro F1 Test:  0.6754888010540184
Micro F1 Train Dev:  0.7559667673716011
Micro F1 Test:  0.791860465116279
Weighted Recall Train Dev:  0.7559667673716012
Weighted Recall Test:  0.791860465116279
Macro Recall Train Dev:  0.6803275401069518
Macro Recall Test:  0.6551747311827957
Micro Recall Train Dev:  0.7559667673716012
Micro Recall Test:  0.791860465116279
Confusion Matrix Train Dev: 
[[8008  832]
 [2399 2001]]
Confusion Matrix Test: 
[[598  22]
 [157  83]]
